In [1]:
from generator import *
from models.attention import Seq2SeqAttn
from models.rnn_only import Seq2SeqRNN
import json
from tqdm import tqdm

# load data
with open('data/selected_scored_essay.json', 'r', encoding='utf8') as f:
    raw = json.load(f)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
enc_char2id_path = 'ckpt/enc_char2id.json'
dec_char2id_path = 'ckpt/dec_char2id.json'
wordlib_path = 'data/WordLib.json'
model_class = Seq2SeqRNN
need_scr = True
bidirectional = False
# RNN
model_path = 'ckpt/RNN.pt'
rnn_type = 'RNN'
rnn = Generator(enc_char2id_path, dec_char2id_path, model_path, wordlib_path, device, need_scr, model_class, bidirectional, rnn_type)
# GRU
model_path = 'ckpt/GRU.pt'
rnn_type = 'GRU'
gru = Generator(enc_char2id_path, dec_char2id_path, model_path, wordlib_path, device, need_scr, model_class, bidirectional, rnn_type)
# LSTM
model_path = 'ckpt/LSTM.pt'
rnn_type = 'LSTM'
lstm = Generator(enc_char2id_path, dec_char2id_path, model_path, wordlib_path, device, need_scr, model_class, bidirectional, rnn_type)
# BiLSTM
model_path = 'ckpt/BiLSTM.pt'
rnn_type = 'LSTM'
bidirectional = True
bilstm = Generator(enc_char2id_path, dec_char2id_path, model_path, wordlib_path, device, need_scr, model_class, bidirectional, rnn_type)
# attention
model_class = Seq2SeqAttn
need_scr = False
model_path = 'ckpt/attention.pt'
rnn_type = None
attention = Generator(enc_char2id_path, dec_char2id_path, model_path, wordlib_path, device, need_scr, model_class, bidirectional, rnn_type)
# attention_s
model_class = Seq2SeqAttn
need_scr = True
model_path = 'ckpt/attention_s.pt'
rnn_type = None
attention_s = Generator(enc_char2id_path, dec_char2id_path, model_path, wordlib_path, device, need_scr, model_class, bidirectional, rnn_type)

In [2]:
txt, scr = raw[0]
attention_s.generate(txt, scr)[1]

{'采分点': ['(F)文笔朴实，气势连贯，文风格外朴实老练'],
 '扣分点': ['(R)笔力欠缺，内容不甚充实，把握不甚准确',
  '(C)语句<unk>嗦重复',
  '(L)抒情勉强，文章繁冗，提法有点不妥',
  '(L)论述不充分，道理不太简明，意境不甚高远']}

In [3]:
res = []
i = 1
for txt, scr in tqdm(raw):
    res.append({
        'id': i,
        'essay': txt,
        'RNN': rnn.generate(txt, scr)[1],
        'GRU': gru.generate(txt, scr)[1],
        'LSTM': lstm.generate(txt, scr)[1],
        'BiLSTM': bilstm.generate(txt, scr)[1],
        'our model(-S)': attention.generate(txt, scr)[1],
        'our model(+S)': attention_s.generate(txt, scr)[1]
    })
    i += 1

100%|██████████| 20/20 [00:11<00:00,  1.82it/s]


In [4]:
with open('data/res.json', 'w', encoding='utf8') as f:
    json.dump(res, f, ensure_ascii=False, indent=4)

In [11]:
with open('data/res.json', 'r') as f:
    res = json.load(f)[5]
def show_res(res):
    if type(res) in [str, int]:
        print(res)
    elif type(res) == list:
        for x in res:
            print(x)
    else:
        for key in res:
            print(key + '：')
            show_res(res[key])
show_res(res)

id：
6
essay：
划船比赛大家应该都看过吧，但是你有看过乒乓球划船比赛吗？应该很少看到吧，今天我就看到了一场非常激烈的划船比赛。
上课了，同学们飞快地跑到教室里坐好，等待老师发号施令，这时只见吴老师拿几个杯子放在讲台上说：中间这一组第一排的同学把书收下去。说话间，吴老师便把杯子放到桌子上，把水倒在里面。好，现在我们开始选人上来比赛。老师讲道。比赛选手选好了，我是裁判员，同学们正在做准备工作，有的同学在指导他们办法，有时同学在那练习，还有的同学在那推推打打。而我则在那儿为他们加水，做服务。
比赛开始了，一局的同学们开始比赛，第一局是对战赛，第一位同学是唐俊峰，唐俊峰肥的挺可爱的，他弯着腰，屁股一翘，嘟起小嘴巴，用力一吹，水花四溅，乒乓球飞到天空，掉到另一水杯里，他再用力一吹，乒乓球好像不愿意离开小水屋，只是水在那儿晃动，唐俊峰火昌三丈了，用上吃奶的劲一吹，乒乓球跳了出来，滚到地上，我便把乒乓球捡起来放到原来掉下来的地方，唐俊峰准备再吹时，只听对方组一片欢呼声，我们组已经赢了。
这节课非常的有趣，我从来没有玩过这种游戏，我非常喜欢，以后有机会，一定再来玩。
RNN：
采分点：
(R)诉述详细，描述分外具体
(C)表达清楚，结构略微不太明朗
(F)作文得当，描述十分真实
扣分点：
(L)描写薄弱含糊
(L)意境不够悠远
GRU：
采分点：
(R)表达丰富，描写尤为精准逼真
(C)特点突出，言语简洁，感情极为突出
(F)描述统一，描写有序，风格有点不统一
扣分点：
(L)形式呆板，新意一般，写法有些陈旧
(L)论据不够充分，水平不太高，道理不太简明
LSTM：
采分点：
(R)语言准确，文章繁冗，文章不甚真实
(C)规则简洁，思路尤其曲折，文字尤为精准精当
(F)书写规范，条理欠佳，内容不太深刻
扣分点：
(L)文章繁冗，描写不甚充分，表达不太充分
(L)情怀散漫，感受有些不深刻
BiLSTM：
采分点：
(R)形容准确，文章分外充实具体
(C)态度鲜明，切入点巧妙，文笔极其独到，语言非常新鲜
(F)笔法老练，语言连贯，表述熟练直白熟练
扣分点：
(L)形式呆板，新意一般，描写有些冗杂平淡
(L)思想不甚深刻，道理不甚简明，论据不够充分
our model(-S)：
采分点：
(R)描写精准，描述极其入神，文章极其精炼，描述较为细腻
(C)作文清晰，结构紧密，文章清